# Q-PART C-15


In [61]:
from pulp import *
import pyomo.environ as pe
import logging

logging.getLogger('pyomo.core').setLevel(logging.ERROR)



In [62]:
from pyomo.environ import *
from math import pi
import warnings
warnings.filterwarnings('ignore')
m = ConcreteModel()

m.a = pe.Set(initialize=[1, 2, 3, 4])
m.demand = pe.Var(m.a, bounds=(1e-20,2500))
m.disc= pe.Var(m.a, bounds=(.1,.6))
m.saleVal = pe.Var(m.a, bounds=(.6*606,606))
m.inventory = pe.Var(m.a, bounds=(1e-20,2500))

m.age = Param([1,2,3,4],  initialize={1:96, 2:97, 3:98, 4:99})

def inv_rule(m, i):
    if i ==1:
        return 2476 >= m.inventory[i] - m.demand[i]
    else:
        return m.inventory[i-1] >= m.inventory[i] - m.demand[i]
    
m.c1 = pe.Constraint(m.a, rule=inv_rule)

def demand_rule(m, i):
    if i==1:
        return m.demand[i] <= ((4.41125 + 3.89091 * m.disc[i] - 0.18602 * (m.age[i]**.5-1)/.5 
                                -3.19977 * .579 
                                + 0.56468 * (48**.2 -1)/.2+ 0.80126)*.2+1)**5
    else:
        return m.demand[i] <= ((4.41125 + 3.89091 * m.disc[i] - 0.18602 * (m.age[i]**.5-1)/.5 
                                -3.19977 * m.disc[i-1] 
                                + 0.56468 * (m.demand[i-1]**.2 -1)/.2+ 0.80126)*.2+1)**5
        
m.c2 = pe.Constraint(m.a, rule=demand_rule)

def disc_rule(m, i):
    if i ==1:
        return m.disc[i] >= .1
    else:
        return m.disc[i] >= m.disc[i-1]
    
m.c3 = pe.Constraint(m.a, rule=disc_rule)


def d_i_rule(m, i):
    if i ==1:
        return m.demand[i] <= 2476
    else:
        return m.demand[i] <= m.inventory[i-1]
    
m.c4 = pe.Constraint(m.a, rule=d_i_rule)

def d_i_rule2(m):
    return sum([m.demand[i]  for i in [1,2,3,4]]) <= 2476
m.c5 = pe.Constraint(rule=d_i_rule2)


m.o = Objective(expr= 
                (sum([m.demand[i]*(1-m.disc[i])*606 for i in [1,2,3,4]])), sense=maximize)

solver = SolverFactory('ipopt')
status = solver.solve(m)

print("Status = %s" % status.solver.termination_condition)

for i in [1,2,3,4]:
    print("%s = %f" % (m.demand, value(m.demand[i])))
    
for i in [1,2,3,4]:
    print("%s = %f" % (m.disc, value(m.disc[i])))
    
#print("%s = %f" % (m.disc, value(m.y)))
print("Objective = %f" % value(m.o))

Status = optimal
demand = 16.659688
demand = 22.480693
demand = 33.425632
demand = 41.424099
disc = 0.100000
disc = 0.150475
disc = 0.315890
disc = 0.458748
Objective = 48103.861923


In [63]:
#.pprint()

##### Analysis
We have used non-linear optimization to derive the outcome.
- The discounts offerred as per the plan:
    - Week 1 - 10%
    - Week 2 - 15%
    - Week 3 - 31%
    - Week 4 - 41%

- Units sold would be:
    - Week 1 - 16
    - Week 2 - 22
    - Week 3 - 33
    - Week 4 - 41
    
- Revenue from Sales: 48103, which is higher than 41302 which he made in reality.

Store would have sold less units 112, compared to the 159 the shop sold, but the revenue generated would have been close to 7000 more (without taking into consideration the extra profit he would make by selling the additional units for 60% discount approximately 11000). So in total, he would have made an extra 18000 by using the above optimization strategy.

# Q-3



In [89]:
data = ["Service was very good. Excellent breakfast in beautiful restaurant included in price. I was happy there and extended my stay for extra two days.",
        "Really helpful staff, the room was clean, beds really comfortable. Great roof top restaurant with yummy food and very friendly staff.",
        "Good location. The Cleanliness part was superb.",
        "I stayed for two days in deluxe A/C room (Room no. 404). I think it is renovated recently. Staff behaviour, room cleanliness all are fine.",
        "The room and public spaces were infested with mosquitoes. I killed a dozen or so in my room prior to sleeping but still woke up covered in bites.",
        "Unfriendly staff with no care for guests.",
        "Very worst and bad experience, Service I got from the hotel reception is too worst and typical.",
        "Good location but the staff was unfriendly"
       ]

data = pd.DataFrame(data)
data.columns = ['text']
data['sentiment'] = [1,1,1,1,0,0,0,-3]

In [90]:
def clean_text(x):
    splchars = re.compile(r'[^A-Za-z ]',re.IGNORECASE)
    x = splchars.sub('', x)
    x = word_tokenize(x.lower())
    x = [w for w in x if w not in stopwords]
    return(' '.join(x))

data.fillna('NA', inplace=True)
data['text_clean'] = data['text'].apply(lambda x: clean_text(x.lower()))
  
count_vec_v1 = CountVectorizer(stop_words=stopwords, 
                               ngram_range=(1,2), max_features=5000)
                               
count_vec_dict = count_vec_v1.fit(data.text_clean)
reviews_text_vec = count_vec_v1.transform(data.text)
df_reviews = pd.DataFrame(reviews_text_vec.toarray())
df_reviews.columns = count_vec_dict.get_feature_names()

print("Data with all possible 1,2 Grams.")
df_reviews.head()                               

Data with all possible 1,2 Grams.


,ac,ac room,bad,bad experience,beautiful,beautiful restaurant,beds,beds really,behaviour,behaviour room,bites,breakfast,breakfast beautiful,care,care guests,clean,clean beds,cleanliness,cleanliness fine,cleanliness part,comfortable,comfortable great,covered,covered bites,days,days deluxe,deluxe,deluxe ac,dozen,dozen room,excellent,excellent breakfast,experience,experience service,extended,extended stay,extra,extra two,fine,food,food friendly,friendly,friendly staff,good,good excellent,good location,got,got hotel,great,great roof,guests,happy,happy extended,helpful,helpful staff,hotel,hotel reception,included,included price,infested,infested mosquitoes,killed,killed dozen,location,location cleanliness,location staff,mosquitoes,mosquitoes killed,part,part superb,price,price happy,prior,prior sleeping,public,public spaces,really,really comfortable,really helpful,recently,recently staff,reception,reception worst,renovated,renovated recently,restaurant,restaurant included,restaurant yummy,roof,roof top,room,room clean,room cleanliness,room prior,room public,room room,room think,service,service good,service got,sleeping,sleeping still,spaces,spaces infested,staff,staff behaviour,staff care,staff room,staff unfriendly,stay,stay extra,stayed,stayed two,still,still woke,superb,think,think renovated,top,top restaurant,two,two days,typical,unfriendly,unfriendly staff,woke,woke covered,worst,worst bad,worst typical,yummy,yummy food
0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,3,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [91]:
columns = ['Beautiful', 'Good Service', 'Good Location', 'Superb', 'Cleanliness', 'Mosquitoes', 
           'Unfriendly', 'bad experience']
columns = [c.lower() for c in columns]

print("Phrases / Words to consider")
columns

Phrases / Words to consider


['beautiful',
 'good service',
 'good location',
 'superb',
 'cleanliness',
 'mosquitoes',
 'unfriendly',
 'bad experience']

In [92]:
df_reviews = df_reviews[list(set(columns).intersection(set(df_reviews.columns)))]

print("Train data")
df_reviews

Train data


,superb,beautiful,unfriendly,good location,cleanliness,bad experience,mosquitoes
0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0
2,1,0,0,1,1,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1
5,0,0,1,0,0,0,0
6,0,0,0,0,0,1,0
7,0,0,1,1,0,0,0


In [93]:
y_train = data.sentiment[:-1]
X_train = df_reviews.iloc[:-1,]
X_test = pd.DataFrame(df_reviews.iloc[-1,:]).T

print("Test Data")
X_test


Test Data


,superb,beautiful,unfriendly,good location,cleanliness,bad experience,mosquitoes
7,0,0,1,1,0,0,0


In [94]:
print("As we can see from above 'good service' is not available in the training set / test set and hence it will not be considered for model building")

print("building Naive Bayes Model..")

bayes_clf = BernoulliNB()
bayes_clf.fit(X_train, y_train)



As we can see from above 'good service' is not available in the training set / test set and hence it will not be considered for model building
building Naive Bayes Model..


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [95]:
pred = bayes_clf.predict_proba(X_test)
print("Probability of Negative Sentiment is : {}".format(pred[0, 0]))
print("Probability of Positive Sentiment is : {}".format(pred[0, 1]))

Probability of Negative Sentiment is : 0.5633083058452802
Probability of Positive Sentiment is : 0.43669169415472003
